In [2]:
import cv2
import numpy as np

In [3]:
back_sub = cv2.createBackgroundSubtractorMOG2()
# back_sub = cv2.createBackgroundSubtractorKNN()

vc = cv2.VideoCapture('segment01.mp4')

if not vc.isOpened:
    print('Unable to open video file!')
    exit(0)

In [3]:
while True:
    ret, frame = vc.read()
    if frame is None:
        break
    
    # Sharpen frame
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])  # sharpen kernal
    sharpened_frame = cv2.filter2D(frame, -1, kernel)

    # Absolute Sobel detector
    dx = cv2.Sobel(sharpened_frame, cv2.CV_64F, 1, 0)
    dy = cv2.Sobel(sharpened_frame, cv2.CV_64F, 0, 1)
    abs_dx = cv2.convertScaleAbs(dx)
    abs_dy = cv2.convertScaleAbs(dy)
    frame_edge = cv2.addWeighted(abs_dx, 0.5, abs_dy, 0.5, 0)

    # Apply background subtractor
    fg_mask = back_sub.apply(frame_edge)

    cv2.rectangle(frame, (10, 2), (100, 20), (255,255,255), -1)
    cv2.putText(frame, str(vc.get(cv2.CAP_PROP_POS_FRAMES)), (15,15), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0))
    
    cv2.imshow('Frame', frame)
    cv2.imshow('FG mask', fg_mask)

    keyboard = cv2.waitKey(30)
    if keyboard =='q' or keyboard == 27:
        break